In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Engineering para el Modelo Parte 2

## Cargar datos

In [75]:
file_path = '../data/processed/completo_df_v1.parquet'

completo_df_cargado = pd.read_parquet(file_path)

print(f"Archivo cargado exitosamente desde: {file_path}")
completo_df_cargado.info()

Archivo cargado exitosamente desde: ../data/processed/completo_df_v1.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27002 entries, 0 to 27001
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   COD_PERSONA          27002 non-null  int64  
 1   COD_ALUMNO           27002 non-null  int64  
 2   SEXO                 27002 non-null  object 
 3   PER_INGRESO          27002 non-null  object 
 4   ESTADO_CIVIL         27002 non-null  object 
 5   TIPO_COLEGIO         27002 non-null  object 
 6   PTJE_INGRESO         27002 non-null  float64
 7   ESTADO               27002 non-null  object 
 8   SEM_CURSADOS         27002 non-null  int64  
 9   CANT_RESERVAS        27002 non-null  int64  
 10  PER_MATRICULA        27002 non-null  object 
 11  COD_CURSO            27002 non-null  object 
 12  CURSO                27002 non-null  object 
 13  CREDITOS             27002 non-null  int64  
 14  TIPO_CURS

In [76]:
import pandas as pd
import numpy as np

def parsear_periodo(periodo):
    """Convierte 'YYYY-0X' a un entero ordenable YYYY*10 + X."""
    año, ciclo = periodo.split('-')
    return int(año) * 10 + int(ciclo[-1])

def crear_tabla_estadisticos(df):
    """
    Crea una tabla con clave (COD_ALUMNO, PER_MATRICULA)
    y estadísticos históricos calculados usando datos previos del mismo alumno.
    """
    df = df.copy()
    
    # Normalizar el periodo
    df["PER_INT"] = df["PER_MATRICULA"].apply(parsear_periodo)

    # Ordenar por alumno y periodo
    df = df.sort_values(["COD_ALUMNO", "PER_INT"]).reset_index(drop=True)

    # Lista de resultados
    registros = []

    # Agrupar por alumno
    for cod, grupo in df.groupby("COD_ALUMNO", sort=False):
        grupo = grupo.sort_values("PER_INT")
        for periodo in grupo["PER_MATRICULA"].unique():
            per_int = parsear_periodo(periodo)
            anteriores = grupo[grupo["PER_INT"] < per_int]
            
            # Si no hay historial previo, inicializamos con ceros
            if len(anteriores) == 0:
                registros.append({
                    "COD_ALUMNO": cod,
                    "PER_MATRICULA": periodo,
                    "PROM_POND_HIST": 0,
                    "NOTA_MAX_HIST": 0,
                    "NOTA_MIN_HIST": 0,
                    "NOTA_MEDIAN_HIST": 0,
                    "NOTA_Q1_HIST": 0,
                    "NOTA_Q3_HIST": 0,
                    "ASIST_PROM_HIST": 0,
                    "CRED_APROB_HIST": 0
                })
                continue

            # Estadísticos
            notas = anteriores["NOTA"]
            asist = anteriores["PRCTJE_INASISTENCIA"]
            cred_aprob = anteriores.loc[anteriores["APROBO"] == "S", "CREDITOS"]

            ponderado = np.average(notas, weights=anteriores["CREDITOS"])
            registros.append({
                "COD_ALUMNO": cod,
                "PER_MATRICULA": periodo,
                "PROM_POND_HIST": ponderado,
                "NOTA_MAX_HIST": notas.max(),
                "NOTA_MIN_HIST": notas.min(),
                "NOTA_MEDIAN_HIST": notas.median(),
                "NOTA_Q1_HIST": notas.quantile(0.25),
                "NOTA_Q3_HIST": notas.quantile(0.75),
                "ASIST_PROM_HIST": asist.mean(),
                "CRED_APROB_HIST": cred_aprob.sum()
            })
    
    # Convertir a DataFrame
    tabla_estadisticos = pd.DataFrame(registros)
    return tabla_estadisticos

In [77]:
def unir_estadisticos(df_base, tabla_hist):
    """
    Une la tabla de estadísticos históricos con la tabla principal
    usando las columnas COD_ALUMNO y PER_MATRICULA.
    """
    df_base = df_base.copy()
    tabla_hist = tabla_hist.copy()

    # Evitar duplicados en la tabla histórica (por seguridad)
    tabla_hist = tabla_hist.drop_duplicates(subset=["COD_ALUMNO", "PER_MATRICULA"])

    # Hacemos un left join: mantenemos todas las filas de la tabla base
    df_merged = df_base.merge(
        tabla_hist,
        on=["COD_ALUMNO", "PER_MATRICULA"],
        how="left",
        validate="many_to_one"  # Cada alumno-periodo debe tener solo una fila en tabla_hist
    )

    # Si hay valores faltantes (por ejemplo primer ciclo), los llenamos con 0
    cols_hist = [
        "PROM_POND_HIST", "NOTA_MAX_HIST", "NOTA_MIN_HIST", "NOTA_MEDIAN_HIST",
        "NOTA_Q1_HIST", "NOTA_Q3_HIST", "ASIST_PROM_HIST", "CRED_APROB_HIST"
    ]
    df_merged[cols_hist] = df_merged[cols_hist].fillna(0)

    return df_merged


In [78]:
import pandas as pd
import numpy as np

def crear_tabla_estadisticos_cluster(df):
    """
    Crea una tabla con estadísticas históricas por COD_ALUMNO y CLUSTER_CURSO.
    Se calculan las estadísticas usando datos donde PER_MATRICULA sea menor
    (histórico) para cada alumno y cluster.
    """
    df = df.copy()

    # Convertir PER_MATRICULA (YYYY-0X) a formato comparable
    df["PER_NUM"] = df["PER_MATRICULA"].apply(lambda x: int(x.split("-")[0]) * 10 + int(x.split("-")[1]))

    registros = []

    for alumno, sub_df in df.groupby("COD_ALUMNO"):
        for cluster, sub_cluster in sub_df.groupby("CLUSTER_CURSO"):
            sub_cluster = sub_cluster.sort_values("PER_NUM")

            for i, row in sub_cluster.iterrows():
                historial = sub_cluster[sub_cluster["PER_NUM"] < row["PER_NUM"]]
                if len(historial) == 0:
                    continue

                prom_pond = historial["PONDERADO"].mean()
                nota_max = historial["NOTA"].max()
                nota_min = historial["NOTA"].min()
                nota_median = historial["NOTA"].median()
                nota_q1 = historial["NOTA"].quantile(0.25)
                nota_q3 = historial["NOTA"].quantile(0.75)
                asist_prom = (1 - historial["PRCTJE_INASISTENCIA"] / 100).mean()
                cred_aprob = historial.loc[historial["APROBO"] == "S", "CREDITOS"].sum()

                registros.append({
                    "COD_ALUMNO": alumno,
                    "CLUSTER_CURSO": cluster,
                    "PER_MATRICULA": row["PER_MATRICULA"],
                    "PROM_POND_CLUSTER_HIST": prom_pond,
                    "NOTA_MAX_CLUSTER_HIST": nota_max,
                    "NOTA_MIN_CLUSTER_HIST": nota_min,
                    "NOTA_MEDIAN_CLUSTER_HIST": nota_median,
                    "NOTA_Q1_CLUSTER_HIST": nota_q1,
                    "NOTA_Q3_CLUSTER_HIST": nota_q3,
                    "ASIST_PROM_CLUSTER_HIST": asist_prom,
                    "CRED_APROB_CLUSTER_HIST": cred_aprob
                })

    return pd.DataFrame(registros)


In [79]:
def unir_estadisticos_cluster(df_base, tabla_cluster_hist):
    """
    Une la tabla de estadísticos históricos por cluster con la tabla principal.
    Se hace merge por COD_ALUMNO, CLUSTER_CURSO y PER_MATRICULA.
    """
    df_base = df_base.copy()
    tabla_cluster_hist = tabla_cluster_hist.copy()

    tabla_cluster_hist = tabla_cluster_hist.drop_duplicates(subset=["COD_ALUMNO", "CLUSTER_CURSO", "PER_MATRICULA"])

    df_merged = df_base.merge(
        tabla_cluster_hist,
        on=["COD_ALUMNO", "CLUSTER_CURSO", "PER_MATRICULA"],
        how="left",
        validate="many_to_one"
    )

    cols_hist = [
        "PROM_POND_CLUSTER_HIST", "NOTA_MAX_CLUSTER_HIST", "NOTA_MIN_CLUSTER_HIST",
        "NOTA_MEDIAN_CLUSTER_HIST", "NOTA_Q1_CLUSTER_HIST", "NOTA_Q3_CLUSTER_HIST",
        "ASIST_PROM_CLUSTER_HIST", "CRED_APROB_CLUSTER_HIST"
    ]
    df_merged[cols_hist] = df_merged[cols_hist].fillna(0)

    return df_merged


In [80]:
def generar_estadisticos_completos(df):
    hist_general = crear_tabla_estadisticos(df)
    df = unir_estadisticos(df, hist_general)
    hist_cluster = crear_tabla_estadisticos_cluster(df)
    df = unir_estadisticos_cluster(df, hist_cluster)
    return df

In [81]:
# Supongamos que ya cargaste tu DataFrame completo_df_cargado
df_resultado = generar_estadisticos_completos(completo_df_cargado)

# Vista previa
print(df_resultado.head())

   COD_PERSONA  COD_ALUMNO SEXO PER_INGRESO ESTADO_CIVIL  \
0        15788       15798    M     2011-01            S   
1        14933       14938    M     2011-01            S   
2        14933       14938    M     2011-01            S   
3        14933       14938    M     2011-01            S   
4        22813       16125    M     2011-02            S   

                  TIPO_COLEGIO  PTJE_INGRESO    ESTADO  SEM_CURSADOS  \
0           Privada Particular         120.0  Retirado             2   
1           Privada Particular         135.0  Separado             6   
2           Privada Particular         135.0  Separado             6   
3           Privada Particular         135.0  Separado             6   
4  Pública otro Sector Público         105.0  Retirado             6   

   CANT_RESERVAS  ... ASIST_PROM_HIST CRED_APROB_HIST PROM_POND_CLUSTER_HIST  \
0              0  ...            30.0               6                 9.2391   
1              1  ...            30.0         

In [82]:
df_resultado

,COD_PERSONA,COD_ALUMNO,SEXO,PER_INGRESO,ESTADO_CIVIL,TIPO_COLEGIO,PTJE_INGRESO,ESTADO,SEM_CURSADOS,CANT_RESERVAS,...,ASIST_PROM_HIST,CRED_APROB_HIST,PROM_POND_CLUSTER_HIST,NOTA_MAX_CLUSTER_HIST,NOTA_MIN_CLUSTER_HIST,NOTA_MEDIAN_CLUSTER_HIST,NOTA_Q1_CLUSTER_HIST,NOTA_Q3_CLUSTER_HIST,ASIST_PROM_CLUSTER_HIST,CRED_APROB_CLUSTER_HIST
0,15788,15798,M,2011-01,S,Privada Particular,120.0,Retirado,2,0,...,30.000000,6,9.23910,1.63,1.63,1.63,1.63,1.63,0.700,0.0
1,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,30.000000,5,10.86360,11.38,11.38,11.38,11.38,11.38,0.700,0.0
2,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,30.000000,5,10.86360,10.13,10.13,10.13,10.13,10.13,0.700,0.0
3,14933,14938,M,2011-01,S,Privada Particular,135.0,Separado,6,1,...,30.000000,5,10.86360,7.75,7.75,7.75,7.75,7.75,0.700,0.0
4,22813,16125,M,2011-02,S,Pública otro Sector Público,105.0,Retirado,6,0,...,0.000000,0,0.00000,0.00,0.00,0.00,0.00,0.00,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26997,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,30.000000,17,14.58710,14.68,14.68,14.68,14.68,14.68,0.700,4.0
26998,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,30.000000,17,14.58710,13.52,13.52,13.52,13.52,13.52,0.700,4.0
26999,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,30.000000,17,14.58710,13.52,13.52,13.52,13.52,13.52,0.700,4.0
27000,33223,32540,M,2017-01,S,Privada Particular,113.0,Retirado,7,1,...,30.000000,17,14.58710,4.10,4.10,4.10,4.10,4.10,0.700,0.0


In [83]:
df_resultado[df_resultado['COD_ALUMNO'] == 28283]

,COD_PERSONA,COD_ALUMNO,SEXO,PER_INGRESO,ESTADO_CIVIL,TIPO_COLEGIO,PTJE_INGRESO,ESTADO,SEM_CURSADOS,CANT_RESERVAS,...,ASIST_PROM_HIST,CRED_APROB_HIST,PROM_POND_CLUSTER_HIST,NOTA_MAX_CLUSTER_HIST,NOTA_MIN_CLUSTER_HIST,NOTA_MEDIAN_CLUSTER_HIST,NOTA_Q1_CLUSTER_HIST,NOTA_Q3_CLUSTER_HIST,ASIST_PROM_CLUSTER_HIST,CRED_APROB_CLUSTER_HIST
4887,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,30.000000,33,12.708567,14.75,8.60,13.460,11.0300,14.1050,0.700000,7.0
4888,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,30.000000,33,12.921650,12.48,10.83,11.655,11.2425,12.0675,0.700000,5.0
4889,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,30.000000,33,12.708567,14.75,8.60,13.460,11.0300,14.1050,0.700000,7.0
4890,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,30.000000,33,12.708567,16.05,15.50,15.970,15.7350,16.0100,0.700000,7.0
4891,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,30.000000,33,12.921650,12.48,10.83,11.655,11.2425,12.0675,0.700000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25335,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,29.869565,140,15.440900,17.12,16.24,16.680,16.4600,16.9000,0.700000,8.0
25336,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,29.869565,140,13.703827,16.61,11.72,15.420,15.2300,16.0100,0.701818,26.0
25551,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,29.812500,92,13.873500,16.50,9.05,12.280,11.4600,14.5300,0.702222,24.0
25955,28739,28283,F,2016-01,S,Privada Particular,128.0,Regular,14,0,...,29.812500,92,0.000000,0.00,0.00,0.000,0.0000,0.0000,0.000000,0.0


In [84]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27002 entries, 0 to 27001
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   COD_PERSONA               27002 non-null  int64  
 1   COD_ALUMNO                27002 non-null  int64  
 2   SEXO                      27002 non-null  object 
 3   PER_INGRESO               27002 non-null  object 
 4   ESTADO_CIVIL              27002 non-null  object 
 5   TIPO_COLEGIO              27002 non-null  object 
 6   PTJE_INGRESO              27002 non-null  float64
 7   ESTADO                    27002 non-null  object 
 8   SEM_CURSADOS              27002 non-null  int64  
 9   CANT_RESERVAS             27002 non-null  int64  
 10  PER_MATRICULA             27002 non-null  object 
 11  COD_CURSO                 27002 non-null  object 
 12  CURSO                     27002 non-null  object 
 13  CREDITOS                  27002 non-null  int64  
 14  TIPO_C

In [85]:
df_resultado.sort_values(by=['COD_PERSONA', 'PER_MATRICULA'])
df_agrupado = df_resultado.groupby(['COD_PERSONA', 'PER_MATRICULA'])

In [97]:
import pandas as pd

# --- 1. Asegurar formato de periodo ordenable ---
df_resultado = df_resultado.copy()
df_resultado["PER_NUM"] = df_resultado["PER_MATRICULA"].apply(
    lambda x: int(x.split("-")[0]) * 10 + int(x.split("-")[1])
)

# --- 2. Ordenar por alumno y periodo ---
df_resultado = df_resultado.sort_values(by=["COD_ALUMNO", "PER_NUM"])

# --- 3. Calcular acumulado de horas de inasistencia ---
df_resultado["HRS_INASISTENCIA_ACUM"] = (
    df_resultado.groupby("COD_ALUMNO")["HRS_INASISTENCIA"]
    .cumsum()
)

# --- 4. Aplicar shift: obtener acumulado HASTA el ciclo anterior ---
df_resultado["HRS_INASISTENCIA_ACUM_PASADO"] = (
    df_resultado.groupby("COD_ALUMNO")["HRS_INASISTENCIA_ACUM"]
    .shift(1)
    .fillna(0)
)

# --- 5. Crear tabla agrupada por alumno y periodo ---
df_acum = (
    df_resultado
    .groupby(["COD_ALUMNO", "PER_MATRICULA"], as_index=False)
    .agg({
        "HRS_INASISTENCIA": "sum",
        "HRS_INASISTENCIA_ACUM_PASADO": "max"
    })
    .rename(columns={
        "HRS_INASISTENCIA": "HRS_INASISTENCIA_PERIODO",
        "HRS_INASISTENCIA_ACUM_PASADO": "HRS_INASISTENCIA_ACUM_PASADO"
    })
)

# --- 6. Unir con la tabla original ---
df_final = df_resultado.merge(
    df_acum,
    on=["COD_ALUMNO", "PER_MATRICULA"],
    how="left"
)

# --- 7. Limpieza opcional ---
df_final = df_final.drop(columns=["PER_NUM"])


In [98]:
import pandas as pd

# --- 1. Copiar y convertir el periodo a un número ordenable ---
df_estado = df_resultado.copy()
df_estado["PER_NUM"] = df_estado["PER_MATRICULA"].apply(
    lambda x: int(x.split("-")[0]) * 10 + int(x.split("-")[1])
)

# --- 2. Ordenar por alumno y periodo ---
df_estado = df_estado.sort_values(by=["COD_PERSONA", "PER_NUM"])

# --- 3. Agrupar por alumno y ciclo para tener un solo estado por periodo ---
df_estado_ciclo = (
    df_estado.groupby(["COD_PERSONA", "PER_MATRICULA"], as_index=False)
    .agg({"ESTADO": "first"})  # usa "last" si prefieres el último registro del ciclo
)

# --- 4. Calcular el estado del ciclo anterior ---
df_estado_ciclo["ESTADO_PASADO"] = (
    df_estado_ciclo.groupby("COD_PERSONA")["ESTADO"]
    .shift(1)
    .fillna("Regular")  # 👈 valor por defecto
)

# --- 5. Unir con el dataframe original ---
df_final = df_final.merge(
    df_estado_ciclo[["COD_PERSONA", "PER_MATRICULA", "ESTADO_PASADO"]],
    on=["COD_PERSONA", "PER_MATRICULA"],
    how="left"
)

# --- 6. Limpieza opcional ---
df_final = df_final.drop(columns=["PER_NUM"], errors="ignore")


In [99]:
df_final

,COD_PERSONA,COD_ALUMNO,SEXO,PER_INGRESO,ESTADO_CIVIL,TIPO_COLEGIO,PTJE_INGRESO,ESTADO,SEM_CURSADOS,CANT_RESERVAS,...,NOTA_MEDIAN_CLUSTER_HIST,NOTA_Q1_CLUSTER_HIST,NOTA_Q3_CLUSTER_HIST,ASIST_PROM_CLUSTER_HIST,CRED_APROB_CLUSTER_HIST,HRS_INASISTENCIA_ACUM,HRS_INASISTENCIA_ACUM_PASADO_x,HRS_INASISTENCIA_PERIODO,HRS_INASISTENCIA_ACUM_PASADO_y,ESTADO_PASADO
0,7085,14334,M,2011-01,S,Pública otro Sector Público,120.0,Separado,16,0,...,0.0,0.0,0.0,0.0,0.0,24,0.0,33,30.0,Regular
1,7085,14334,M,2011-01,S,Pública otro Sector Público,120.0,Separado,16,0,...,0.0,0.0,0.0,0.0,0.0,30,24.0,33,30.0,Regular
2,7085,14334,M,2011-01,S,Pública otro Sector Público,120.0,Separado,16,0,...,0.0,0.0,0.0,0.0,0.0,30,30.0,33,30.0,Regular
3,7085,14334,M,2011-01,S,Pública otro Sector Público,120.0,Separado,16,0,...,0.0,0.0,0.0,0.0,0.0,33,30.0,33,30.0,Regular
4,7085,14334,M,2011-01,S,Pública otro Sector Público,120.0,Separado,16,0,...,14.2,14.2,14.2,0.7,4.0,39,33.0,36,57.0,Separado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26997,40398,43662,M,2020-02,S,Privada Parroquial,170.0,Separado,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,Separado
26998,40398,43662,M,2020-02,S,Privada Parroquial,170.0,Separado,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,Separado
26999,40398,43662,M,2020-02,S,Privada Parroquial,170.0,Separado,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,Separado
27000,40398,43662,M,2020-02,S,Privada Parroquial,170.0,Separado,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,Separado


In [101]:
df_final.to_parquet('../data/processed/completo_df_v2.parquet', index=False)